In [1]:
from pathlib import Path
from time import time
import datetime

import numpy as np
import pandas as pd

from scipy.stats import spearmanr

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

In [2]:
# Global Settings

# Directories
model_dataDir = 'DATA/MODELDATA/'

# Time
START = 2020
END = 2022


# Helpers
sns.set_style('whitegrid')
np.random.seed(42)
idx = pd.IndexSlice


In [7]:
# Helper Functions
def get_trade_prices(start, stop):
    model_mdf = pd.read_csv(model_dataDir + 'ModelData.csv')
    model_mdf.rename(columns={'Unnamed: 0': 'Dates'}, inplace=True)
    model_mdf['Dates'] = pd.to_datetime(model_mdf['Dates'])
    model_mdf.set_index(['Dates', 'Coin'], inplace=True)
    #model_mdf = model_mdf.loc[:,'Open']
    #model_mdf = model_mdf.loc[idx[tickers, str(START):str(END)], 'Open']
    prices = model_mdf.unstack('Coin').sort_index().shift(-1).tz_localize('UTC')
    
    return prices


def get_predictions(predictions, prices):

    prices = prices.loc[:,'Open':'High']
    best_alpha = predictions.groupby('alpha').apply(lambda x: spearmanr(x.actuals, x.predicted)[0]).idxmax()
    predictions = predictions[predictions.alpha == best_alpha]
    predictions.index.names = ['Coin', 'Dates']
    tickers = predictions.index.get_level_values('Coin').unique()
    
    start = predictions.index.get_level_values('Dates').min().strftime('%Y-%m-%d')
    stop = (predictions.index.get_level_values('Dates').max() + pd.DateOffset(1)).strftime('%Y-%m-%d')
    prices = prices.sort_index().loc[idx[tickers, START:END], :]
    predictions = predictions.loc[predictions.alpha == best_alpha, ['predicted']]
    
    return predictions.join(prices, how='right')

SyntaxError: can't assign to function call (3089280832.py, line 21)

In [6]:
# Import data

# Prices
trade_prices = get_trade_prices(2020, 2022)

# Benchmark
trade_prices = trade_prices.loc[: , 'Open']
btc_bench = trade_prices.bitcoin.pct_change()

# Predictions
lr_predictions = pd.read_hdf(f'{model_dataDir}lr_model_predictions.h5', 'lr/predictions')
ridge_predictions = pd.read_hdf(f'{model_dataDir}Ridge_Predictions.h5', 'ridge/predictions')
lasso_predictions = pd.read_hdf(f'{model_dataDir}Lasso_Predictions.h5', 'lasso/predictions')

# Backtest Prices and Predictions
data = get_predictions(ridge_predictions, trade_prices)


AttributeError: 'str' object has no attribute 'strftime'

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
btc_bench.tail()

In [ ]:
trade_prices.head()

In [ ]:
ridge_predictions.info()

In [ ]:
# Vectorized backtest

In [ ]:
# Event driven backtest with Backtestester